In [1]:
# -*- coding: utf-8 -*-
"""
Second Jupyter notebook as a working copy for when the first one is busy scraping

"""

import pandas as pd
import matplotlib.pyplot as plt
#import csv

import sqlite3
from godwin import Scraper, Database
import requests
from lxml import html

db = Database('Godwin.db')